In [0]:
from pyspark.sql.functions import to_date, col, round, current_timestamp
import sys
from time import time
from pyspark.sql.utils import AnalysisException

In [0]:
start_time = time()

In [0]:
checkpoint_path = "/Volumes/fz_catalog/landing/_checkpoints/products_silver/"
target_table = "fz_catalog.silver.products"

In [0]:
# Read streaming data from the bronze Delta table, validate schema contract, and safely project required columns.

expected_columns = {"product_id", "category", "sub_category"}
table_name = "fz_catalog.bronze.products"

try:
    # Read stream from Delta table
    df_bronze = (
        spark.readStream
             .format("delta")
             .table(table_name)
    )

    # Validate schema contract
    missing_columns = expected_columns - set(df_bronze.columns)
    if missing_columns:
        raise ValueError(
            f"Bronze table schema violation for {table_name}. "
            f"Missing column(s): {sorted(missing_columns)}"
        )

    # Safe projection after validation
    df = df_bronze.select(*expected_columns)

except AnalysisException as e:
    # Table not found, permission issue, or Delta metadata problem
    raise AnalysisException(
        f"Failed to read streaming Bronze table {table_name}: {e}"
    )

except ValueError:
    # Explicit schema contract violation
    raise

In [0]:
# Add a created_date column with the current timestamp to the DataFrame for Silver table enrichment
df_silver = df.withColumn("created_date", current_timestamp())

In [0]:
try:
    (
        df_silver.writeStream
            .format("delta")
            .outputMode("append")
            .option("checkpointLocation", checkpoint_path)
            .trigger(availableNow=True)
            .toTable(target_table)
    )

except AnalysisException as e:
    # Delta table issues, permission problems, schema mismatch, etc.
    raise AnalysisException(
        f"Failed to write streaming data to Silver table {target_table}: {e}"
    )

except Exception as e:
    # Catch-all for unexpected streaming runtime failures
    raise RuntimeError(
        f"Unexpected error while writing to Silver table {target_table}: {e}"
    )

In [0]:
end_time = time()

In [0]:
dbutils.notebook.exit(f"Run Time: {end_time - start_time} seconds")